In [73]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [87]:
df_udacity= pd.ExcelFile(r'data/UdacityABtesting.xlsx')

df_experiment = pd.read_excel(df_udacity,'Experiment')
df_control = pd.read_excel(df_udacity,'Control')

In [75]:
df_soccer = pd.read_excel(r'data/SoccerData.xlsx')

In [76]:
df_soccer.head()

,league,teams_no,date,home_team,away_team,home_odd,draw_odd,away_odd,h_played,a_played,...,a_final,h_half,a_half,BTS,H1H,A1H,H2H,A2H,Unnamed: 79,Unnamed: 80
0,France-Ligue 2,20,2016-01-08,Ajaccio,Le Havre,2.63,2.97,3.24,19,19,...,1.0,NaN,NaN,1,0.0,0.0,0.0,0.0,NaN,NaN
1,France-Ligue 2,20,2016-01-08,Auxerre,Valenciennes,1.98,3.25,4.69,19,19,...,1.0,NaN,NaN,1,0.0,0.0,0.0,0.0,NaN,NaN
2,France-Ligue 2,20,2016-01-08,Bourg Peronnas,Creteil,2.00,3.52,4.14,19,19,...,1.0,NaN,NaN,1,0.0,0.0,1.0,0.0,NaN,NaN
3,France-Ligue 2,20,2016-01-08,Chamois,Dijon,2.82,3.06,2.91,19,19,...,2.0,NaN,NaN,1,0.0,0.0,0.0,0.0,NaN,NaN
4,France-Ligue 2,20,2016-01-08,Clermont,Evian,2.10,3.21,4.20,19,19,...,1.0,NaN,NaN,1,0.0,0.0,1.0,0.0,NaN,NaN


In [77]:
df_soccer.shape

(24830, 81)

## AB Testing

In [88]:
df_experiment.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7716,686,105.0,34.0
1,"Sun, Oct 12",9288,785,116.0,91.0
2,"Mon, Oct 13",10480,884,145.0,79.0
3,"Tue, Oct 14",9867,827,138.0,92.0
4,"Wed, Oct 15",9793,832,140.0,94.0


In [89]:
df_control.head()

,Date,Pageviews,Clicks,Enrollments,Payments
0,"Sat, Oct 11",7723,687,134.0,70.0
1,"Sun, Oct 12",9102,779,147.0,70.0
2,"Mon, Oct 13",10511,909,167.0,95.0
3,"Tue, Oct 14",9871,836,156.0,105.0
4,"Wed, Oct 15",10014,837,163.0,64.0


## Answers to Quiz

1. Enrolments to payment ratio
2. 37 days each
3. 28 missing values in control and experiment
4. It will be a uniform probability distribution. This is because, 
    the student will either click the button or not
    and each event has same likelihood of occuring

#### Number of days of experiment in experiment group and control group

In [90]:
df_control.Date.nunique(),df_experiment.Date.nunique()

(37, 37)

#### Missing values in control and experiment

In [91]:
df_control.isna().sum(),df_experiment.isna().sum()

(Date            0
 Pageviews       0
 Clicks          0
 Enrollments    14
 Payments       14
 dtype: int64, Date            0
 Pageviews       0
 Clicks          0
 Enrollments    14
 Payments       14
 dtype: int64)

## Machine learning Tasks

### Distinguish control from experiment dataframe

In [92]:
df_control['id'] = 0
df_experiment['id'] = 1

In [93]:
df_experiment.head()

,Date,Pageviews,Clicks,Enrollments,Payments,id
0,"Sat, Oct 11",7716,686,105.0,34.0,1
1,"Sun, Oct 12",9288,785,116.0,91.0,1
2,"Mon, Oct 13",10480,884,145.0,79.0,1
3,"Tue, Oct 14",9867,827,138.0,92.0,1
4,"Wed, Oct 15",9793,832,140.0,94.0,1


### Combine dataframe

In [94]:
df_combined = pd.concat([df_control,df_experiment])

In [95]:
df_combined.head(2)

,Date,Pageviews,Clicks,Enrollments,Payments,id
0,"Sat, Oct 11",7723,687,134.0,70.0,0
1,"Sun, Oct 12",9102,779,147.0,70.0,0


In [96]:
df_combined.tail(3)

,Date,Pageviews,Clicks,Enrollments,Payments,id
34,"Fri, Nov 14",9304,770,NaN,NaN,1
35,"Sat, Nov 15",8668,724,NaN,NaN,1
36,"Sun, Nov 16",8988,710,NaN,NaN,1


### Extract day of week

In [97]:
df_combined['day_of_week'] = df_combined.Date.apply(lambda x: x.split(',')[0])

In [98]:
df_combined.head()

,Date,Pageviews,Clicks,Enrollments,Payments,id,day_of_week
0,"Sat, Oct 11",7723,687,134.0,70.0,0,Sat
1,"Sun, Oct 12",9102,779,147.0,70.0,0,Sun
2,"Mon, Oct 13",10511,909,167.0,95.0,0,Mon
3,"Tue, Oct 14",9871,836,156.0,105.0,0,Tue
4,"Wed, Oct 15",10014,837,163.0,64.0,0,Wed


### Drop Date and Payment

In [99]:
df_combined.drop(['Date','Payments'],axis=1,inplace=True)

In [100]:
df_combined.head(3)

,Pageviews,Clicks,Enrollments,id,day_of_week
0,7723,687,134.0,0,Sat
1,9102,779,147.0,0,Sun
2,10511,909,167.0,0,Mon


### Drop rows with null values

In [101]:
df_combined.dropna(how='any',axis=0,inplace=True)

### Shuffle dataframe inplace and reset index

In [102]:
df_combined.sample(frac=1).reset_index(drop=True)

,Pageviews,Clicks,Enrollments,id,day_of_week
0,9088,780,127.0,1,Fri
1,9345,734,167.0,0,Thu
2,8448,695,142.0,1,Sat
3,10014,837,163.0,0,Wed
4,9308,728,207.0,1,Thu
5,8459,691,131.0,0,Sun
6,9500,788,129.0,1,Thu
7,9402,697,194.0,1,Fri
8,8715,722,182.0,1,Fri
9,10496,860,153.0,1,Mon


### Data cleaning

#### convert categorical to numerical

In [104]:
df_combined.head()

,Pageviews,Clicks,Enrollments,id,day_of_week
0,7723,687,134.0,0,Sat
1,9102,779,147.0,0,Sun
2,10511,909,167.0,0,Mon
3,9871,836,156.0,0,Tue
4,10014,837,163.0,0,Wed


In [107]:
df_combined = pd.get_dummies(df_combined, drop_first=True)

In [109]:
df_combined.head()

,Pageviews,Clicks,Enrollments,id,day_of_week_Mon,day_of_week_Sat,day_of_week_Sun,day_of_week_Thu,day_of_week_Tue,day_of_week_Wed
0,7723,687,134.0,0,0,1,0,0,0,0
1,9102,779,147.0,0,0,0,1,0,0,0
2,10511,909,167.0,0,1,0,0,0,0,0
3,9871,836,156.0,0,0,0,0,0,1,0
4,10014,837,163.0,0,0,0,0,0,0,1


In [111]:
df_combined.Enrollments.min(),df_combined.Enrollments.max()

(94.0, 233.0)

In [112]:
X = df_combined.drop('Enrollments',axis=1)
y = df_combined['Enrollments'].copy()

### Standardize data 

In [114]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X.values)
df_x = pd.DataFrame(x_scaled)

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [115]:
df_x

,0,1,2,3,4,5,6,7,8
0,0.089391,0.198556,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.515929,0.530686,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.951748,1.000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.753789,0.736462,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.798020,0.740072,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.691618,0.689531,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6,0.486854,0.418773,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,0.317043,0.212996,0.0,0.0,0.0,1.0,0.0,0.0,0.0
9,1.000000,0.826715,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### Train Test Split

In [116]:
from sklearn.model_selection import train_test_split

In [117]:
X_train, X_test, y_train, y_test = train_test_split(df_x, y, test_size=0.2)

### Fit models

#### Linear regression

In [120]:
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from math import sqrt

In [121]:
regressor = LinearRegression()  
regressor.fit(X_train, y_train) #training the algorithm

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [122]:
y_pred = regressor.predict(X_test)

###### rmse

In [124]:
rms = sqrt(mean_squared_error(y_test, y_pred))

rms

24.709792860884605

In [126]:
mse = mean_squared_error(y_test,y_pred)

mse

610.5738632278237

#### Decision Tree

In [127]:
from sklearn.tree import DecisionTreeRegressor

In [128]:
d_regressor = DecisionTreeRegressor()
d_regressor.fit(X_train,y_train)

y_pred = d_regressor.predict(X_test)

#### rmse and mse

In [129]:
mean_squared_error(y_test,y_pred),sqrt(mean_squared_error(y_test, y_pred))

(991.5, 31.488092987667578)

#### XG Boost

In [131]:
!pip install xgboost

    100% |████████████████████████████████| 686kB 214kB/s ta 0:00:01
  Stored in directory: /Users/socrates/Library/Caches/pip/wheels/e9/48/4d/de4187b5270dff71d3697c5a7857a1e2d9a0c63a28b3462eeb
Successfully built xgboost


In [133]:
import xgboost as xgb

In [138]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)
xg_reg.fit(X_train,y_train)

y_pred = xg_reg.predict(X_test)

[22:07:45] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


#### rmse and mse

In [139]:
mean_squared_error(y_test,y_pred),sqrt(mean_squared_error(y_test, y_pred))

(4174.147209561476, 64.60764048904338)